# Video2X Codespace Notebook

Adapted from the original [Video2X Colab Notebook](https://github.com/k4yt3x/video2x) for GitHub Codespace environment.

**Features:**
- 🎯 RealESRGAN and libplacebo filters
- 📁 Local file system (no Google Drive dependency)
- ⚡ CPU/GPU processing support
- 🔧 Interactive configuration

**Note:** This runs in a Codespace environment with local storage.

In [ ]:
# Step 0: System Check
import subprocess

print("🔍 Checking system resources...")

# Check GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ GPU detected:")
        print(result.stdout)
    else:
        print("⚠️ No GPU detected - will use CPU processing")
except FileNotFoundError:
    print("⚠️ nvidia-smi not found - CPU processing only")

# Check Video2X
try:
    result = subprocess.run(['video2x', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\n✅ Video2X is available:")
        print(result.stdout)
    else:
        print("\n❌ Video2X not found - run setup first")
except FileNotFoundError:
    print("\n❌ Video2X not installed - run setup script")

In [ ]:
# Step 1: File Management
import pathlib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set up local paths
workspace_path = pathlib.Path('/workspaces/video2x-codespace')
input_dir = workspace_path / 'input'
output_dir = workspace_path / 'output'
temp_dir = workspace_path / 'temp'

# Ensure directories exist
for dir_path in [input_dir, output_dir, temp_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

input_file = None

# Check for existing files
input_files = [f for f in input_dir.iterdir() if f.is_file()]
if input_files:
    input_file = input_files[0]
    print(f"✅ Found input file: {input_file.name}")
else:
    print("📁 No files found in input directory")
    print(f"📍 Upload files to: {input_dir}")
    print("💡 Use VS Code file explorer to drag and drop videos")

In [ ]:
# Step 2: Configuration
import ipywidgets as widgets

# Default configuration
filter_type = 'realesrgan'
model = 'realesr-animevideov3'
scale = 2
codec = 'libx264'
crf = 20

# Interactive widgets
filter_widget = widgets.Dropdown(
    options=[('RealESRGAN', 'realesrgan'), ('libplacebo', 'libplacebo')],
    value=filter_type,
    description='Filter:'
)

scale_widget = widgets.IntSlider(
    value=scale, min=2, max=4, step=1, description='Scale:'
)

print("⚙️ Configuration:")
display(filter_widget)
display(scale_widget)

print(f"🎯 Filter: {filter_type}")
print(f"🤖 Model: {model}")
print(f"📏 Scale: {scale}x")

In [ ]:
# Step 3: Video Processing
import subprocess

if input_file is None:
    print("❌ No input file selected. Upload a video first.")
else:
    output_file = output_dir / f"{input_file.stem}_enhanced_{scale}x{input_file.suffix}"
    
    if output_file.exists():
        print(f"⚠️ Output already exists: {output_file.name}")
    else:
        print(f"🎬 Processing: {input_file.name}")
        print(f"📤 Output: {output_file.name}")
        
        # Build Video2X command
        command = [
            'video2x',
            '--input', str(input_file),
            '--output', str(output_file),
            '--processor', filter_type,
            '--scaling-factor', str(scale),
            '--realesrgan-model', model,
            '--codec', codec
        ]
        
        print("🚀 Starting processing...")
        result = subprocess.run(command)
        
        if result.returncode == 0 and output_file.exists():
            size_mb = output_file.stat().st_size / (1024*1024)
            print(f"✅ Success! Output: {size_mb:.1f} MB")
        else:
            print("❌ Processing failed")

In [ ]:
# Step 4: Results
output_files = [f for f in output_dir.iterdir() if f.is_file()]

if output_files:
    print(f"📁 Found {len(output_files)} processed files:")
    for f in output_files:
        size_mb = f.stat().st_size / (1024*1024)
        print(f"  📄 {f.name} ({size_mb:.1f} MB)")
else:
    print("📭 No processed files found")

print("\n💡 Download Instructions:")
print("1. Use VS Code file explorer")
print(f"2. Navigate to: {output_dir}")
print("3. Right-click file → Download")